In [1]:
import pandas as pd
random_state = 20
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [2]:
random_state = 20
def read_dataset(dataset, model_name):
    import pandas as pd
    import numpy as np
    import pickle
    from IPython.display import clear_output
    from sklearn import preprocessing
    clear_output()
    #datasets = ['TC02_20news', 'TC14_wos']
    #models_names = ["all-MiniLM-L6-v2","all-MiniLM-L12-v2"]
    if model_name != "TFIDF":
        embeddings_path = f'Arquivos TCC/embeddings/sbert/{model_name}_{dataset}.pkl'
        with open(embeddings_path, "rb") as fIn:
            stored_data = pickle.load(fIn)
        embeddings = stored_data['embeddings']
        labels = stored_data['labels']
        labels_enc = preprocessing.LabelEncoder().fit_transform(labels)
        unique, counts = np.unique(stored_data['labels'], return_counts=True)
        #Split hold-out test data
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(embeddings, labels_enc, test_size=0.10, random_state=random_state,stratify=labels_enc)
        return (X_train, X_test, y_train, y_test)
    else:
        df = pd.read_csv(f'Arquivos TCC/datasets/{dataset}.csv', sep=';')
        df = df.dropna()
        textos = df['text'].values.tolist()
        labels = df['label']
        labels_enc = preprocessing.LabelEncoder().fit_transform(labels)
        #Split hold-out test data
        from sklearn.model_selection import train_test_split
        textos_train, textos_test, y_train, y_test = train_test_split(textos, labels_enc, test_size=0.10, random_state=random_state,stratify=labels_enc)
        return (textos_train, textos_test, y_train, y_test)

def get_opt_params(pipeline):
    import ast
    df_cv = pd.read_csv('Arquivos TCC/perfomance_cv.csv')
    df_cv = df_cv.drop_duplicates(subset=['pipeline'], keep='last')
    params = df_cv.loc[df_cv['pipeline']==pipeline , 'params']
    params = ast.literal_eval(params.values[0])
    return params

def eval_classifier(pipeline):
    import numpy as np
    import pandas as pd
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.svm import LinearSVC
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.pipeline import make_pipeline
    from sklearn.metrics import balanced_accuracy_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import confusion_matrix
    #pipeline = TC02_20news+all-MiniLM-L6-v2+KNN
    dataset, model_name ,classifier =tuple(pipeline.split('+'))
    params = get_opt_params(pipeline=pipeline) 
    print(params)
    X_train, X_test, y_train, y_test = read_dataset(dataset=dataset, model_name=model_name)
    if model_name != "TFIDF":
        if classifier == 'KNN':
            clf = KNeighborsClassifier(**params)
        if classifier == 'LR':
            clf = LogisticRegression(**params)
        if classifier == 'SVC':
            clf = LinearSVC(**params)
        if classifier == 'RF':
            clf = RandomForestClassifier(**params)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        balanced_acc = balanced_accuracy_score(y_test,y_pred)
        recall = recall_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        conf_matrix = confusion_matrix(y_test, y_pred)
        pd.DataFrame(conf_matrix).to_csv(f"Arquivos TCC/confusion_matrix/{pipeline}.csv", header=None, index=None,float_format='%.1f')
        print(color.BOLD+"A acurácia balanceada nos dados de teste: {0:.4f}".format(balanced_acc)+color.END)
        df_test_pred = pd.DataFrame()
        df_test_pred['test_label'] = y_test
        df_test_pred['pred_label'] = y_pred
        df_test_pred.to_csv(f"Arquivos TCC/test_set_predictions/{pipeline}.csv", index=None,float_format='%.1f')
        return (balanced_acc, recall, precision)
    else:
        if classifier == 'KNN':
            clf = KNeighborsClassifier()
        if classifier == 'LR':
            clf = LogisticRegression()
        if classifier == 'SVC':
            clf = LinearSVC()
        if classifier == 'RF':
            clf = RandomForestClassifier()
        pipeline_clf = make_pipeline(TfidfVectorizer(stop_words='english',smooth_idf=True), clf)
        pipeline_clf.set_params(**params)
        textos_train = X_train
        textos_test = X_test
        pipeline_clf.fit(textos_train, y_train)
        y_pred = pipeline_clf.predict(textos_test)
        balanced_acc = balanced_accuracy_score(y_test,y_pred)
        recall = recall_score(y_test, y_pred, average='weighted')
        precision = precision_score(y_test, y_pred, average='weighted')
        conf_matrix = confusion_matrix(y_test, y_pred)
        pd.DataFrame(conf_matrix).to_csv(f"Arquivos TCC/confusion_matrix/{pipeline}.csv", header=None, index=None,float_format='%.1f')
        print(color.BOLD+"A acurácia balanceada nos dados de teste: {0:.4f}".format(balanced_acc)+color.END)
        df_test_pred = pd.DataFrame()
        df_test_pred['test_label'] = y_test
        df_test_pred['pred_label'] = y_pred
        df_test_pred.to_csv(f"Arquivos TCC/test_set_predictions/{pipeline}.csv", index=None,float_format='%.1f')
        return (balanced_acc, recall, precision)


In [3]:
df_cv = pd.read_csv('Arquivos TCC/perfomance_cv.csv')
df_cv = df_cv.drop_duplicates(subset=['pipeline'], keep='last')
for pipe in df_cv['pipeline'].unique()[-5:]:
    params = get_opt_params(pipeline=pipe)
    
type(params)

dict

In [4]:
dataset= 'TC02_20news'
model_name='TFIDF'
X_train, X_test, y_train, y_test = read_dataset(dataset=dataset, model_name=model_name)
print(X_train[0][:10])
print(len(y_test))
print(pd.Series(y_test).value_counts())

and key si
1999
11    100
14    100
0     100
17    100
5     100
1     100
4     100
9     100
15    100
16    100
3     100
7     100
2     100
13    100
12    100
18    100
10    100
8     100
19    100
6      99
dtype: int64


In [5]:
df_cv = pd.read_csv('Arquivos TCC/perfomance_cv.csv')
df_cv = df_cv.drop_duplicates(subset=['pipeline'], keep='last')
#for pipe in df_cv['pipeline'].unique()[:1]:

pipe = 'TC02_20news+TFIDF+SVC'
acc, recall, precision = eval_classifier(pipeline=pipe)
print(pipe,acc, recall)

A acurácia balanceada nos dados de teste: 0.8604
TC02_20news+TFIDF+SVC 0.8604191919191919 0.8604302151075538


In [6]:
eval_dict = {}
for pipe in df_cv['pipeline']:
    eval_dict[pipe] = eval_classifier(pipeline=pipe)

print(eval_dict)

A acurácia balanceada nos dados de teste: 0.8479
{'TC02_20news+all-MiniLM-L6-v2+KNN': (0.8274141414141413, 0.8274137068534267, 0.8294367161512026), 'TC02_20news+all-MiniLM-L6-v2+LR': (0.7763888888888888, 0.7763881940970485, 0.7747964441569772), 'TC02_20news+all-MiniLM-L6-v2+SVC': (0.7879141414141414, 0.7878939469734867, 0.7843601200001915), 'TC02_20news+all-MiniLM-L6-v2+RF': (0.7568535353535354, 0.7568784392196098, 0.752278271425051), 'TC02_20news+all-MiniLM-L12-v2+KNN': (0.8184191919191919, 0.8184092046023012, 0.8200427675302876), 'TC02_20news+all-MiniLM-L12-v2+LR': (0.7733838383838384, 0.7733866933466733, 0.7771961582822929), 'TC02_20news+all-MiniLM-L12-v2+SVC': (0.7778939393939394, 0.7778889444722361, 0.7764327415864273), 'TC02_20news+all-MiniLM-L12-v2+RF': (0.7788888888888889, 0.7788894447223612, 0.7770631423680294), 'TC14_wos+all-MiniLM-L6-v2+KNN': (0.7392962882355963, 0.7493734335839599, 0.7500942171236378), 'TC14_wos+all-MiniLM-L6-v2+LR': (0.7846916674467614, 0.7894736842105263,

In [7]:
eval_dict = {'TC02_20news+all-MiniLM-L6-v2+KNN': (0.8274141414141413, 0.8274137068534267, 0.8294367161512026), 'TC02_20news+all-MiniLM-L6-v2+LR': (0.7763888888888888, 0.7763881940970485, 0.7747964441569772), 'TC02_20news+all-MiniLM-L6-v2+SVC': (0.7879141414141414, 0.7878939469734867, 0.7843601200001915), 'TC02_20news+all-MiniLM-L6-v2+RF': (0.7648939393939393, 0.7648824412206103, 0.7602691280999301), 'TC02_20news+all-MiniLM-L12-v2+KNN': (0.8184191919191919, 0.8184092046023012, 0.8200427675302876), 'TC02_20news+all-MiniLM-L12-v2+LR': (0.7733838383838384, 0.7733866933466733, 0.7771961582822929), 'TC02_20news+all-MiniLM-L12-v2+SVC': (0.7778939393939394, 0.7778889444722361, 0.7764327415864273), 'TC02_20news+all-MiniLM-L12-v2+RF': (0.7718838383838383, 0.7718859429714857, 0.7692920062551857), 'TC14_wos+all-MiniLM-L6-v2+KNN': (0.7392962882355963, 0.7493734335839599, 0.7500942171236378), 'TC14_wos+all-MiniLM-L6-v2+LR': (0.7846916674467614, 0.7894736842105263, 0.7909831611389914), 'TC14_wos+all-MiniLM-L6-v2+SVC': (0.7835047468784679, 0.7878028404344194, 0.7893107948543343), 'TC14_wos+all-MiniLM-L6-v2+RF': (0.7265016047315879, 0.7368421052631579, 0.7380088704902777), 'TC14_wos+all-MiniLM-L12-v2+KNN': (0.7242640876789967, 0.7351712614870509, 0.7388915856243542), 'TC14_wos+all-MiniLM-L12-v2+LR': (0.766612388507014, 0.7719298245614035, 0.7767255620162534), 'TC14_wos+all-MiniLM-L12-v2+SVC': (0.756727109552949, 0.7644110275689223, 0.7655758359621054), 'TC14_wos+all-MiniLM-L12-v2+RF': (0.7280875035502954, 0.7401837928153717, 0.7427305410303153), 'TC02_20news+TFIDF+KNN': (0.7407979797979797, 0.7408704352176088, 0.7564466366873973), 'TC02_20news+TFIDF+LR': (0.8544343434343433, 0.8544272136068034, 0.8563148895347136), 'TC02_20news+TFIDF+SVC': (0.8604191919191919, 0.8604302151075538, 0.8614286630438546), 'TC02_20news+TFIDF+RF': (0.7969040404040404, 0.7968984492246123, 0.7931393533590855), 'TC14_wos+TFIDF+KNN': (0.6987338659497362, 0.7101086048454469, 0.7284659798426261), 'TC14_wos+TFIDF+LR': (0.8101840789698002, 0.8162071846282373, 0.8187984103729264), 'TC14_wos+TFIDF+SVC': (0.822927757798685, 0.8287385129490392, 0.8304343585641185), 'TC14_wos+TFIDF+RF': (0.8430377772332633, 0.8471177944862155, 0.8503830907723287)}

In [8]:
df_eval = pd.DataFrame.from_dict(eval_dict, orient='index', columns=['balanced_acc', 'weighted_recall', 'weighted_precision'])
df_eval.reset_index(inplace=True)
df_eval[['dataset','embedding','ml_alg']] = df_eval['index'].str.split('+',expand=True)
df_eval = df_eval.drop(columns=['index'])
df_eval = df_eval[['dataset', 'embedding','ml_alg','balanced_acc', 'weighted_recall', 'weighted_precision']]
df_eval_20ng = df_eval[df_eval['dataset']=='TC02_20news'].drop(columns=['dataset'])
df_eval_wos = df_eval[df_eval['dataset']=='TC14_wos'].drop(columns=['dataset'])
print(df_eval_20ng.to_latex(index=False,float_format="%.5f", multicolumn=True))

\begin{tabular}{llrrr}
\toprule
        embedding & ml\_alg &  balanced\_acc &  weighted\_recall &  weighted\_precision \\
\midrule
 all-MiniLM-L6-v2 &    KNN &       0.82741 &          0.82741 &             0.82944 \\
 all-MiniLM-L6-v2 &     LR &       0.77639 &          0.77639 &             0.77480 \\
 all-MiniLM-L6-v2 &    SVC &       0.78791 &          0.78789 &             0.78436 \\
 all-MiniLM-L6-v2 &     RF &       0.76489 &          0.76488 &             0.76027 \\
all-MiniLM-L12-v2 &    KNN &       0.81842 &          0.81841 &             0.82004 \\
all-MiniLM-L12-v2 &     LR &       0.77338 &          0.77339 &             0.77720 \\
all-MiniLM-L12-v2 &    SVC &       0.77789 &          0.77789 &             0.77643 \\
all-MiniLM-L12-v2 &     RF &       0.77188 &          0.77189 &             0.76929 \\
            TFIDF &    KNN &       0.74080 &          0.74087 &             0.75645 \\
            TFIDF &     LR &       0.85443 &          0.85443 &             0.85631 \

/tmp/ipykernel_432/3551327500.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_eval_20ng.to_latex(index=False,float_format="%.5f", multicolumn=True))


In [15]:
print(df_eval_wos.to_latex(index=False,float_format="%.5f"))

\begin{tabular}{llrrr}
\toprule
        embedding & ml\_alg &  balanced\_acc &  weighted\_recall &  weighted\_precision \\
\midrule
 all-MiniLM-L6-v2 &    KNN &       0.73930 &          0.74937 &             0.75009 \\
 all-MiniLM-L6-v2 &     LR &       0.78469 &          0.78947 &             0.79098 \\
 all-MiniLM-L6-v2 &    SVC &       0.78350 &          0.78780 &             0.78931 \\
 all-MiniLM-L6-v2 &     RF &       0.72650 &          0.73684 &             0.73801 \\
all-MiniLM-L12-v2 &    KNN &       0.72426 &          0.73517 &             0.73889 \\
all-MiniLM-L12-v2 &     LR &       0.76661 &          0.77193 &             0.77673 \\
all-MiniLM-L12-v2 &    SVC &       0.75673 &          0.76441 &             0.76558 \\
all-MiniLM-L12-v2 &     RF &       0.72809 &          0.74018 &             0.74273 \\
            TFIDF &    KNN &       0.69873 &          0.71011 &             0.72847 \\
            TFIDF &     LR &       0.81018 &          0.81621 &             0.81880 \

/tmp/ipykernel_400/1027491184.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_eval_wos.to_latex(index=False,float_format="%.5f"))


In [28]:



from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
y_true = [0, 1, 2, 0, 1, 2, 3]
y_pred = [0, 2, 1, 0, 3, 1, 1]
print(recall_score(y_true, y_pred, average='weighted'))
print(accuracy_score(y_true, y_pred))
print(balanced_accuracy_score(y_true, y_pred))

0.2857142857142857
0.2857142857142857
0.25
